# Setup and Imports

Same setup as previous notebook:
- Matplotlib inline configuration
- Standard scientific Python libraries (numpy, pandas)
- ObsPy for seismic data processing
- MSNoise API import
- Plot styling configuration

In [ ]:
%matplotlib inline
import datetime
import matplotlib.pyplot as plt
import glob
import os
from obspy import read, UTCDateTime, read_inventory
from obspy.signal import PPSD
import warnings
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib.dates import DateFormatter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
plt.style.use("ggplot")

from msnoise.api import *

# Database Connection and Parameters

Establishes connection to MSNoise database and loads parameters:
- Creates database connection using `connect()`
- Retrieves processing parameters using `get_params()`

These will be used throughout the notebook for dv/v analysis.

In [ ]:
# connect to the database
db = connect()
params = get_params(db)

# Display Job Status

Shows current MSNoise job statistics using `msnoise info -j` command to verify processing status before proceeding with dv/v analysis.

In [ ]:
! msnoise info -j

# DVV Processing Sequence

Three-step process to compute velocity variations:

1. Compute MWCS (Moving Window Cross Spectral):
   - Command: `msnoise cc dvv compute_mwcs`
   - Uses 5 threads (-t 5)
   - Verbose output (-v)

2. CompRelativeTT (Delta Travel Time):
   - Command: `msnoise cc dvv compute_dtt`
   - Same threading and verbosity settings

3. Compute DVV (Relative Velocity Changes):
   - Command: `msnoise cc dvv compute_dvv`
   
This sequence calculates relative velocity changes from the cross-correlation functions.

In [ ]:
! msnoise -v -t 5 cc dvv compute_mwcs

In [ ]:
! msnoise -v -t 5 cc dvv compute_dtt

In [ ]:
! msnoise cc dvv compute_dvv

# Plot DVV Results

Uses MSNoise's plotting functionality to visualize dv/v results:
- Imports dvv plotting function from msnoise.plots
- Creates plots for filter ID 1
- Parameters:
  - `filterid`: Filter selection (e.g., 1)
  - `show=True`: Display plot immediately

Example commands shown for reference:
- Basic plot: `msnoise cc dvv plot dvv -f 1`
- With moving stack: `msnoise cc dvv plot dvv -f 2 -m 2`

In [ ]:
from msnoise.plots.dvv import main as plot_dvv
for filterid in [1, ]:
    plot_dvv(filterid=filterid, show=True)

# Custom DVV Analysis by Station Pairs

Creates detailed plots of dv/v results for individual station pairs:
- Parameters:
  - Component: "EN" (East-North)
  - Multiple filters and moving stack windows
  
Plot configuration:
- 2x2 subplot grid (10x10 inches)
- One subplot per filter
- Multiple station pairs plotted on each subplot
- Y-axis shows dv/v as percentage
- Results grouped by moving stack window

Features:
- Automatic iteration through all station pairs
- Error handling for missing data
- Legend showing station pair combinations
- Filter frequency ranges in titles

In [ ]:
filters = get_filters(db)
comp = "EN"

for mov_stack in params.mov_stack:
    fig, axes = plt.subplots(2,1, figsize=(10,10))
    plt.suptitle(mov_stack)
    for i, filter in enumerate(filters):
        filterid = filter.ref
        col = i % 2
        for sta1, sta2 in get_station_pairs(db):
            row = 0
            plt.sca(axes[col])
            for loc1 in sta1.locs():
                s1 = "%s.%s.%s" % (sta1.net, sta1.sta, loc1)
                for loc2 in sta2.locs():
                    s2 = "%s.%s.%s" % (sta2.net, sta2.sta, loc2)
                    # if s1 == s2:
                        # continue
                    try:
                        dvv = compute_dvv(db, filterid, mov_stack, components=comp, params=params, pairs=[(s1,s2),])
                    except:
                        continue
                    dvv.m *= -100
                    dvv.m.plot(label="%s:%s"%(s1,s2), ax=axes[col])
                    plt.title("Filter %i (%.2f - %.2f Hz) - %s" % (filterid, filter.low, filter.high, comp))
            # plt.axvline(datetime.date(2014,6,21), c='r', ls='--')
            plt.legend()
            plt.xlabel("")
            plt.ylabel("dv/v (%)")
    plt.tight_layout()
    plt.show()